In [ ]:
import numpy as np
from pathlib import Path
import tensorflow as tf

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3))
])

for _ in range(20):
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal'))

cifar10 = tf.keras.datasets.cifar10.load_data()
model.add(tf.keras.layers.Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

(X_full, y_full), (X_test, y_test) = cifar10
X_train = X_full[5000:]
y_train = y_full[5000:]
X_valid = X_full[:5000]  # Fixed: using X_full instead of y_full
y_valid = y_full[:5000]

early_callback = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model.keras", save_best_only=True)
run_index = 1  # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_bn_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_callback, checkpoint_callback, tensorboard_cb]

%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

model.fit(X_train, y_train, 
          epochs=100,
          validation_data=(X_valid, y_valid), 
          callbacks=callbacks)